In [28]:
import cv2
import os
import pprint
import requests

import matplotlib.pyplot as plt
import numpy as np

from azure_cfg import api_key
from skimage import io

%matplotlib inline

In [29]:
subscription_key = api_key

vision_base_url = "https://westcentralus.api.cognitive.microsoft.com/vision/v1.0/"
vision_analyze_url = vision_base_url + "analyze?"

image_dir = './images/'
train_image_dir = './images/train/'
test_image_dir = './images/test'

In [30]:
def extract_high_level_features(image_path):
    headers  = {'Ocp-Apim-Subscription-Key': subscription_key,
                'Content-Type'             : 'application/octet-stream'}
    params   = {'visualFeatures': 'Adult,Categories,Description,Color,Faces,ImageType,Tags'}
    body     = open(image_path, 'rb')

    response = requests.post(vision_analyze_url, headers=headers, params=params, data=body)
    response.raise_for_status()
    data = response.json()
    
    return {
        "clip_art_type"                      : data["imageType"]["clipArtType"],
        "line_drawing_type"                  : data["imageType"]["lineDrawingType"],
        "is_black_and_white"                 : int(data["color"]["isBwImg"]),
        "is_adult_content"                   : int(data["adult"]["isAdultContent"]),
        "adult_score"                        : data["adult"]["adultScore"],
        "is_racy"                            : int(data["adult"]["isRacyContent"]),
        "racy_score"                         : data["adult"]["racyScore"],
        "has_faces"                          : int(len(data["faces"])),
        "num_faces"                          : len(data["faces"]),
        "is_dominant_color_background_black" : int(data["color"]["dominantColorBackground"] == "Black"),
        "is_dominant_color_foreground_black" : int(data["color"]["dominantColorForeground"] == "Black")
    }

In [31]:
width, height = 128, 64
descriptor = cv2.HOGDescriptor(_winSize = (width,height),
                               _blockSize = (16,16),
                               _blockStride = (8,8),
                               _cellSize = (8,8),
                               _nbins = 9)

def extract_low_level_features(image_path, descriptor=descriptor):
    image = io.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (width,height))

    hog = descriptor.compute(image)
    hog = hog.flatten()
    return hog

In [32]:
def get_class_label(image_path):
    pass

In [33]:
def extract_all_features(image_dir):
    for image_path in os.listdir(image_dir):
        path = image_dir + image_path

        high_level_features = extract_high_level_features(path)
        low_level_features = extract_low_level_features(path)

        features = np.array(list(high_level_features.values()))
        features = np.append(features, low_level_features)
        return features

In [27]:
train_features = extract_all_features(train_image_dir)
test_features = extract_all_features(test_image_dir)

FileNotFoundError: [Errno 2] No such file or directory: './images/VizWiz_train_000000000096.jpg'

In [7]:
from pandas import DataFrame
from IPython.display import display

features = np.array(list(high_level_features.values()))
features = np.append(features, low_level_features)

df = DataFrame(data=np.reshape(features, (1,len(features))))
display(df)

,0,1,2,3,4,5,6,7,8,9,...,3781,3782,3783,3784,3785,3786,3787,3788,3789,3790
0,0.0,0.0,1.0,0.0,0.014061,0.0,0.0,0.013014,1.0,0.0,...,0.018814,0.065769,0.004625,0.089652,0.194971,0.336087,0.197564,0.194937,0.123565,0.09141
